# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=transformer3",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [3]:
cfg.case.user.num_data_points = 4 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [32] # This is the sequence length

cfg.case.data.tokenizer = "word-level"

cfg.case.server.has_external_data = True

cfg.case.server.param_modification.v_length = 16

### Instantiate all parties

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Model architecture transformer3 loaded with 10,702,129 parameters and 480,000 buffers.
Overall this is a data ratio of   83610:1 for target shape [4, 32] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 4

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: transformer3
        model state: default
        public buffers: Fa

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Computing feature distribution before the linear1 layer from external data.
Feature mean is 0.0014430976007133722, feature std is 0.00266657886095345.


In [6]:
user.print(true_user_data)

[CLS] the tower building of the little rock arsenal, also known as u. s. arsenal building, is a building located in macarthur park in downtown little rock,
arkansas. built in 1 8 4 0, it was part of little rock's first military installation. since its decommissioning, the tower building has housed two museums
. it was home to the arkansas museum of natural history and antiquities from 1 9 4 2 to 1 9 9 7 and the macarthur museum of arkansas military history since
2 0 0 1. it has also been the headquarters of the little rock [UNK] club since 1 8 9 4. [SEP] [CLS] the building receives its name from its


# Reconstruct user data

In [10]:
attacker.cfg.sentence_algorithm ="k-means"

In [11]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[0, 1, 2, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 19, 22, 25, 26, 29, 31, 32, 35, 38, 40, 43, 50, 56, 62, 63, 64, 72, 108, 184], [5, 5, 5, 6, 291, 310, 400, 494, 566, 649, 652, 846, 926, 940, 993, 1084, 1495, 1936, 2195, 2971, 3688, 5231, 5470, 6084, 6489, 8107, 8323, 18637, 21489, 21964, 22724, 24378], [5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 11, 11, 11, 12, 17, 19, 19, 22, 22, 22, 31, 35, 38, 291, 566, 566, 1084, 1084, 1084, 1936, 1936], [5, 6, 7, 7, 8, 11, 12, 14, 16, 17, 19, 22, 22, 31, 35, 38, 40, 43, 50, 62, 291, 310, 494, 566, 652, 846, 1084, 1936, 3688, 6084, 6489, 18637]] through strategy decoder-bias.
Recovered 115 embeddings with positional data from imprinted layer.
Assigned [30, 29, 27, 29] breached embeddings to each sentence.
tensor([0.7090, 0.6305, 0.6499, 0.6770, 0.7245, 0.7442, 0.1139, 0.1157, 0.1157,
        0.1140, 0.1150, 0.1154, 0.1141, 0.1153, 0.1147, 0.1138, 0.1154, 0.1143,
        0.1149, 0.1137, 0.1139, 0.1146, 0.1152, 0.1145, 0.1149, 0.1142, 0.1141,
  

In [8]:
reconstructed_user_data, stats = attacker.reconstruct_single_sentence([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[0, 1, 2, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 19, 22, 25, 26, 29, 31, 32, 35, 38, 40, 43, 50, 56, 62, 63, 64, 72, 108, 184], [5, 5, 5, 6, 291, 310, 400, 494, 566, 649, 652, 846, 926, 940, 993, 1084, 1495, 1936, 2195, 2971, 3688, 5231, 5470, 6084, 6489, 8107, 8323, 18637, 21489, 21964, 22724, 24378], [5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 11, 11, 11, 12, 17, 19, 19, 22, 22, 22, 31, 35, 38, 291, 566, 566, 1084, 1084, 1084, 1936, 1936], [5, 6, 7, 7, 8, 11, 12, 14, 16, 17, 19, 22, 22, 31, 35, 38, 40, 43, 50, 62, 291, 310, 494, 566, 652, 846, 1084, 1936, 3688, 6084, 6489, 18637]] through strategy decoder-bias.
Recovered 115 embeddings with positional data from imprinted layer.
little little in two it of building receives tower 2 macarthur it military [CLS] its 4 the rock rock 0 arkansas to [UNK] name park home decommissioning 9. antiquities 1 1
history little in 8 also since since been tower natural macarthur the,, its 4 u, rock it arkansas has is the known and. 9. 9 1 first
hi

In [9]:
reconstructed_user_data, stats = attacker.reconstruct2([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[0, 1, 2, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 19, 22, 25, 26, 29, 31, 32, 35, 38, 40, 43, 50, 56, 62, 63, 64, 72, 108, 184], [5, 5, 5, 6, 291, 310, 400, 494, 566, 649, 652, 846, 926, 940, 993, 1084, 1495, 1936, 2195, 2971, 3688, 5231, 5470, 6084, 6489, 8107, 8323, 18637, 21489, 21964, 22724, 24378], [5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 11, 11, 11, 12, 17, 19, 19, 22, 22, 22, 31, 35, 38, 291, 566, 566, 1084, 1084, 1084, 1936, 1936], [5, 6, 7, 7, 8, 11, 12, 14, 16, 17, 19, 22, 22, 31, 35, 38, 40, 43, 50, 62, 291, 310, 494, 566, 652, 846, 1084, 1936, 3688, 6084, 6489, 18637]] through strategy decoder-bias.
Recovered 115 tokens with positional data from imprinted layer.


IndexError: list index out of range